<p>Автор:<br>Крашенинников Егор<br>Версия python: 3.5.1</p>

<h1 align="center">Отборочные задания GoTo ML</h1>

<p>
<strong>Панель навигации:</strong><br>
<a href="#task1">Задание 1</a><br>
<a href="#task2">Задание 2</a><br>
<a href="#task3">Задание 3</a>
</p>

<h2>Описание задания</h2>

<p>В этом отчете разбираются вступительные задания школы <b>GoTo 2016</b>. Задания представляют собой реальную задачу с которой столкнулись специалисты по анализу данных – компании <span style="color: blue";>E-Contenta</span> при разработке рекомендательного движка для приложения <span style="color: blue";>Tviz.tv</span> и несколько простых аналитических задач. Для их решения предложен <a href="https://yadi.sk/d/SqsWFcpds9rTL" target="_blank">пакет данных</a>, состоящий из фаилов:</p>
<ol>
<li><b>train_likes.csv</b> - выборка о том, какому зрителю какие фильмы понравились.</li>
<li><b>items.json</b> - описание фильмов.</li>
<li><b>schedule.csv</b> - расписание фильмов.</li>
</ol>


<h2>Работа с данными:</h2>

In [1]:
import json
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

with open('items.json') as f:
    items_dicts = json.load(f)   #to list of dicts

train_likes_df = pd.read_csv('train_likes.csv')   #to data frame

<h2>Вводим переменные, которые будем использовать больше, чем в одном задании:</h2> 
<p><b>length_columns</b> - высота <b>dataframe</b>-а (<b>train_likes</b>), считаем высоту произвольной колонки, это не принципиально, потому что они одинаковы. </p>
<p><b>my_dict_IdGenres</b> - словарь, в котором будут пары - <span style="color: green";>id</span> фильма и его жанр, как ключ и значение соответственно.</p>

<p>(id, выделенное зеленым, означает id фильма; в будущем встретится id, выделенное красным, что будет относиться к id канала)</p>

In [2]:
length_columns = train_likes_df['item_id'].count()

my_dict_IdGenres = {}                                #conversion (film's id: its genre)
for i in range(len(items_dicts)):
    my_dict_IdGenres[items_dicts[i]['id']] = items_dicts[i]['genre']

<a name="task1"></a>
<h2>Задание 1</h2>
<h3>Описание задания</h3>
<ol>
    <li>Вычислить среднее количество пользовательских лайков на одном канале.</li>
    <li>Посчитать количество фильмов, имеющих &ge;5 лайков.</li>
<ol>

<h3>Часть I</h3>
<p>Среднее количество лайков на канал можно вычислить разделив количество всех лайков на число различных каналов.
Количество всех лайков равно высоте <b>dataframe</b>-а (<b>train_likes</b>), а это и есть значение переменной <b>length_columns</b>. Количество различных каналов можно узнать, применив к колонке <em>channel</em> функцию <span style="color: blue";>nunique()</span>. Результат округляем с точностью до двух знаков после запятой.</p>

In [3]:
different_channels = train_likes_df['channel'].nunique()  
print(round(length_columns/different_channels,2),'- лайков у одного канала в среднем.')

106.05 - лайков у одного канала в среднем.


<a name="top"></a>
<h3>Часть II</h3>
<p>Чтобы узнать, сколько фильмов получило &ge; 5 лайков, создадим словарь <b>my_dict_IdLikes</b>, в который будем заносить пары - <span style="color: green";>id</span> фильма и сколько раз его лайкнули. Проходя циклом по <span style="color: green";>id</span> каждого фильма в данных (<b>train_likes</b>), смотрим, есть ли в нашем словаре уже такой <span style="color: green";>id</span> (ключ). Если нет - создаем пару в словаре и значение делаем равным единице (только что лайкнули). Если да - увеличиваем значение на 1 по ключу (<span style="color: green";>id</span>).</p>
<p>Создаем переменную-счетчик <b>counter_MoreThanFive</b>, которой будем считать сколько в словаре фильмов, удовлетворяющих условию (&ge;5). Проходим циклом по значениям словаря, если значение &ge;5 - увеличиваем счетчик на 1.</p>

In [4]:
my_dict_IdLikes = {}

#__main_cycle__==================================================================
for i in range(length_columns):
    
    id = train_likes_df['item_id'].loc[i]
    
    if id not in my_dict_IdLikes:
        my_dict_IdLikes[id] = 1
    else:
        my_dict_IdLikes[id] += 1
        
#__output__=====================================================================
counter_MoreThanFive = 0
for value in my_dict_IdLikes.values():
    if value >= 5:
        counter_MoreThanFive += 1

print(counter_MoreThanFive,'- количество фильмов, у которых есть хотя бы 5 лайков.')

4524 - количество фильмов, у которых есть хотя бы 5 лайков.


<em>Created using matplotlib</em><img src="Images/image1.jpg", align="left",width="400">

<p>
<a name="task2"></a>
<strong>Панель навигации:</strong><br>
<a href="#task1">Задание 1</a><br>
<a href="#task3">Задание 3</a>
</p>

<h2>Задание 2</h2>
<h3>Описание задания</h3>
<ol>
    <li>Вычислить сумму лайков для каждого жанра и отдельно - для фильмов, где жанр неизвестен. Выписать последнее в конце строки вывода.</li>
    <li>Вычислить такую же сумму лайков по жанрам для каждого из топ-10 самых залайканных каналов. Указать к какому каналу она относится.</li>
</ol>

<a name="mid"></a>
<h3>Часть I</h3>
<p>Ввод словаря <b>genres</b>, в котором будут пары - жанр и сколько раз его лайкнули. Жанр с номером 10 здесь выполняет роль счетчика фильмов с неизвестным жанром. Далее делаем примерно то же самое, что и в предыдущей части, только вместо <span style="color: green";>id</span> фильма здесь будет номер жанра. </p>

In [5]:
genres = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0}

#__main_cycle__==================================================================
for i in range(length_columns): 
    
    id = train_likes_df['item_id'].loc[i]
    
    if id not in my_dict_IdGenres:
        genres[10] += 1
        
    else:
        genre = my_dict_IdGenres[id]
        genres[genre] += 1

#__output__=====================================================================
for n in sorted(genres.items()):
    print(n[1],end=' ')

14007 28151 14237 5231 2494 6543 4383 1167 27 0 35011 

<em>Created using matplotlib</em><img src="Images/image2.jpg", align="left",width="500">

<h3>Часть II</h3>
<p>Создаем функцию <span style="color: blue";>checking</span>, чтобы не использовать два раза одинаковый кусок кода:</p>

In [6]:
def checking(genre):
    if channel in my_dict_ChannelLikes:
        my_dict_ChannelLikes[channel][genre] += 1
        
    else:
        my_dict_ChannelLikes[channel] = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0}
        my_dict_ChannelLikes[channel][genre] += 1

<h4>Описание работы функции:</h4>
<p>Вызывая эту функцию в цикле, мы проверяем, есть ли в словаре <b>my_dict_ChannelLikes</b> ключ <em>channel</em> (переменная, ссылающаяся на <span style="color: red";>id</span> канала). Если нет, создаем по этому ключу словарь жанров, такой же, как и в предыдущем задании и в нем  увеличиваем значение на 1 по ключу <em>genre</em> (агрумент функции). Если да, то просто увеличиваем значение по этому жанру на 1. </p>

In [7]:
my_dict_ChannelLikes = {}

#__main_cycle__==================================================================
for i in range(length_columns):
    
    id = train_likes_df['item_id'].loc[i]
    channel = train_likes_df['channel'].loc[i]
    
    if id not in my_dict_IdGenres:
        checking(10)
            
    else:
        Genre = my_dict_IdGenres[id]
        checking(Genre)

<p>Посредством вышеприведенного кода получаем следующую структуру данных: <b>my_dict_ChannelLikes</b> = {channel1:{genre1:a,genre2:b...},channel2:{genre1:c,genre2:d...}...}.<br>
Каждый элемент этого словаря (<b>my_dict_ChannelLikes</b>) значит , что на канале <u>channelX</u> фильм с жанром <u>genreY</u> лайкнули <u>K</u> раз.</p>

In [8]:
#__output__=====================================================================     
for n in range(10):
    top_channel = max(my_dict_ChannelLikes.items(), key = lambda x: sum(x[1].values()))
    
    print(str(n+1) + ')' + top_channel[0], end=': ')
    for item in sorted(top_channel[1].items()):
        print(item[1],end = ' ')
        
    print()
    
    del my_dict_ChannelLikes[top_channel[0]]

1)1f0e3dad99908345f7439f8ffabdffc4: 1138 2913 1750 0 13 0 487 0 0 0 2047 
2)c74d97b01eae257e44aa9d5bade97baf: 2364 1689 873 496 366 209 13 0 0 0 2047 
3)98f13708210194c475687be6106a3b84: 1882 1256 1806 3 6 0 46 0 1 0 1079 
4)1ff1de774005f8da13f42943881c655f: 73 157 23 111 70 3500 0 0 0 0 1919 
5)70efdf2ec9b086079795c442636b55fb: 684 99 494 332 213 29 0 0 0 0 2768 
6)c0c7c76d30bd3dcaefc96f40275bdc0a: 50 391 315 0 12 0 1095 0 0 0 1394 
7)f90f2aca5c640289d0a29417bcb63a37: 507 281 76 71 0 0 13 0 21 0 2068 
8)6f4922f45568161a8cdf4ad2299f6d23: 334 380 472 200 257 4 29 0 0 0 1249 
9)4e732ced3463d06de0ca9a15b6153677: 31 1198 282 124 13 0 3 0 0 0 877 
10)5c936263f3428a40227908d5a3847c0b: 1187 546 380 4 0 0 6 0 0 0 360 


<h4>Принцип вывода:</h4>
<ul>
    <li>Из всех сумм значений по словарям жанров ({channelX:<u>{genre1:a,genre2:b...}</u>) выбирается максимальная.</li>
    <li><b>top_channels</b> присваевается кортеж, состоящий из ключа и значения - словаря жанров, в котором сумма значений максимальна.</li>
    <li>Выводится первый элемент <b>top_channels</b> - <span style="color: red";>id</span> канала.</li>
    <li>Второй элемент - словарь жанров. Он сортируется (по возрастанию ключа-жанра) и выписываются все его значения в строку.</li>
    <li>Из <b>my_dict_ChannelLikes</b> удаляется пара с максимальной суммой значений по словарю жанров.</li>
    <li>Все вышеперечисленные операции повторяются 10 раз (топ 10 залайканых каналов).</li>
</ul>

<p>
<a name="task3"></a>
<strong>Панель навигации:</strong><br>
<a href="#task1">Задание 1</a><br>
<a href="#task2">Задание 2</a>
</p>

<h2>Задание 3</h2>
<h3>Описание задания</h3>
<p>На основании имеющихся данных (train_likes.csv, items.json, schedule.csv) нужно проанализировать интересы пользователей и построить алгоритм рекомендации новых фильмов, в точности - создать функцию, которая принимает параметры (id_пользователя, id_фильма, дополнительная информация) и возвращает предсказанную вероятность лайка.</p>

<h3>Ход решения задачи</h3>

<ul>

<li>Сделать <b>dataframe</b> из <b>items.json</b> с колонками (film's) <b>id</b>, <b>year</b>, <b>likes</b>, <b>genre</b> (для genre используем one hot encoding).</li> 

<li>Нормализовать колонку <b>year</b> (0-1).</li>

<li>Сделать функцию <span style="color: blue";>dist</span>, которая считает "расстояние" между двумя заданными фильмами. Чем больше расстояние, тем менее похожи фильмы.</li>

<li>Сделать функцию <span style="color: blue";>get_max_likes</span>, которая для заданного жанра возвращает максимальное количество лайков у фильма этого жанра.</li>

<li>Сделать функцию <span style="color: blue";>liked_films</span>, которая для заданного пользователя возвращает список фильмов, на которые он поставил лайк.</li>

<li>В конце мы делаем функцию <span style="color: blue";>user_likes_film</span>, которая для заданного <b>user_id</b> and <b>film_id</b> возвращает вероятность, что заданному пользователю понравится заданный фильм.</li>

</ul>


<h4>Вероятность считается следующим образом:</h4>

<p>Если дистанция между <u>любым</u> из фильмов, которые пользователь уже лайкнул, и заданным фильмом <b>new_film</b>, меньше 1.1 (экспериментально выявлено что около 10% фильмов попадают в такой диапазон для одного фильма, лайкнутого пользователем), то мы считаем вероятность как:</p>

<p><strong>p = likes_of_new_film/ max_likes_for_new_film_genre</strong>,</p>

<p>где<br> 
<b>max_likes_for_new_film_genre</b> - результат функции <b>get_max_likes</b> для жанра фильма <b>new_film.</b><br>
<b>likes_of_new_film</b> - количество лайков у фильма <b>new_film</b>.</p>

<p>Таким образом, мы с большей вероятностью будем рекомендовать фильмы у которых больше суммарное количество лайков.

Если же дистанция больше 1.1, мы не рекомендуем этот фильм данному пользователю и вероятность равна нулю.

Также мы хотим иногда рекомендовать совершенно рандомные фильмы. С вероятностью 1% мы будем рекомендовать заданный фильм даже если дистанция оказалась больше 1.1.</p>


<h3>Ниже идет код решения и его описание</h3>


<p>У меня не получилось переделать <strong>items.json</strong> в <strong>dataframe</strong> обычными функциями типа <b>pandas.read_json(filename)</b> из-за того, что атрибутов <strong>f_N</strong> слишком много и получается огромнейшее число колонок (и из-за этого мой ноутбук конечно же выдает Memory Error), поэтому я переделал все "вручную" c помощью трех созданных мной функций, обрезав эти атрибуты и создав надежный алгоритм решения задачи без них. </p>
<p>Итак, функция <span style="color: blue";>converting</span> служит для того, чтобы привести колонку (аргумент) <b>column_id</b> из <b>dataframe</b> в <em>one hot encoding</em>.</p>


In [9]:
def converting(dataframe,column_id):

    if column_id in dataframe:
        one_hot_encoding = pd.get_dummies(dataframe[column_id].values)
            
        for column in one_hot_encoding.columns:
            one_hot_encoding.rename(columns={column:column_id + '_' + str(column)}, inplace=True) #rename columns (1 -> genre_1)
            
        dataframe = dataframe.drop(column_id,axis=1)
        dataframe = pd.concat([dataframe,one_hot_encoding],axis=1)
        
    else:
        return dataframe
            
    return dataframe    

<p>Функция <span style="color: blue";>formatting</span> "вырезает" из словаря, переданного в качестве аргумента, лишь пары с ключами <b>id, genre, year</b>, добавляет еще одну пару с ключом <b>likes</b> и количеством лайков на текущем фильме (<span style="color: green";>id</span> из словаря-агрумента) в качестве значения (это значение берется из словаря <b>my_dict_IdLikes</b>/см. задачу <a href="#top">1.2</a>). Затем возвращает новый словарь с этими парами.</p>

In [10]:
def formatting(dic):
    new_dic={}
    for key in dic:
        if key=='id' or key=='genre' or key=='year':
            new_dic[key]=dic[key]
            
    if  new_dic['id'] in my_dict_IdLikes:  
        new_dic['likes']=my_dict_IdLikes[new_dic['id']]
    
    return new_dic

<p>В качестве аргумента функции <span style="color: blue";>open_json</span> ожидается получить список словарей (в дальнейшем это будет <b>items_dicts</b>, созданный в самом начале).</p>
<p>Проходя циклом по всем словарям списка мы применяем к ним <span style="color: blue";>formatting</span> и делаем из каждого словаря <b>dataframe</b> (состоящий из одной строки) и соединяем такие <b>dataframe</b>-ы в один. Затем применяем к результату <span style="color: blue";>converting</span>.</p>

In [11]:
def open_json(file):
    length=len(file)
    df = pd.DataFrame({})
    
    for j in range(length):
        new_dic=formatting(file[j])
        data=pd.DataFrame(new_dic, index=[1])
        df = pd.concat([df,data],ignore_index=True)
        
    return df

<p>С помощью этих функций переделываем <b>items_dicts</b> в <b>dataframe</b>.</p> 
<p>Создаем нормализованную колонку <b>year</b>, используя <b>sklearn</b>. Удаляем старую, добавляем только что созданную  и сохраняем новый <b>dataframe</b> как <em>csv</em>, чтобы не пересоздавать лишний раз (все-таки процесс не быстрый), а просто открывать, если надо.</p>
<p>Далее вывод первых пяти строк новых данных, чтобы увидеть структуру получше.</p>

In [12]:
df = open_json(items_dicts)
df = converting(df,'genre')

normilized_column = df['year'].values 
min_max_scaler = preprocessing.MinMaxScaler()
normilized_column_scaled = min_max_scaler.fit_transform(normilized_column)

df = df.drop('year',axis=1)
df.insert(2,'year',pd.Series(normilized_column_scaled))

df.to_csv('items.csv',index=False)

In [19]:
df.head()

,id,likes,year,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9
0,cd3dc8b6cffb41e4163dcbd857ca87da,160.0,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30d69ba3457fbe63ddf07f88adbca502,176.0,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3d7fdbe4e37f0a7b1409001df2212fc2,378.0,0.991150,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,fe3c9604bff61b422dfaccc167173896,5.0,0.893805,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9c372bc6fa306220ed6fbdd86d2f6bb1,26.0,0.814159,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p>Функция <span style="color: blue";>dist</span> находит расстояние следующим образом:<br>
Для каждого из двух фильмов берется целый ряд из <b>dataframe</b>-а включающий в себя заданный <span style="color: green";>id</span> фильма и затем в этом ряду обрезаются элементы от колонок <b>likes</b> и <b>id</b>. Таким образом, остаются только элементы жанра и года. Затем функцией <span style="color: blue";>np.linalg.norm</span> считается расстояние между двумя векторами.</p>

In [14]:
def dist(film1, film2):
    
    film1_whole_row = df.loc[np.argmax(df['id'] == film1)]
    film1_vec = film1_whole_row[2::]

    film2_whole_row = df.loc[np.argmax(df['id'] == film2)]
    film2_vec = film2_whole_row[2::]

    d = np.linalg.norm(film1_vec-film2_vec)
    return d

dist('cd3dc8b6cffb41e4163dcbd857ca87da', 'fe3c9604bff61b422dfaccc167173896') #example (random films)

1.4181950896264528

<p>Создадим словарь <b>Genres</b>. В девятый элемент (жанр) записываем словарь состоящий из (' ':0), потому что фильмов с девятым жанром лайкнуто не было (при том, что сами фильмы с таким жанром есть. в этом можно убедиться <a href="#mid">здесь</a>), и, таким образом, значение функции <span style="color:blue";>get_max_likes</span> от 9 должно быть нулем. </p>

<p>Перебираем все ключи в словаре <b>my_dict_IdLikes</b>. Если у фильма есть жанр, заносим в <b>Genres</b> по ключу-жанру пару <span style="color: green";>id</span> фильма и количество лайков на нем. </p>
<p>Функция <span style="color:blue";>get_max_likes</span> же просто возвращает максимальное количество лайков, поставленных фильму из этого словаря. </p>

In [15]:
Genres={0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{' ':0}}
for key in my_dict_IdLikes.keys():
    id = key
    if id in my_dict_IdGenres:
        genre = my_dict_IdGenres[id]
        likes = my_dict_IdLikes[key]
        
        Genres[genre][id]=likes
        
def get_max_likes(genre_id):
    return max(Genres[genre_id].values())

get_max_likes(0)

680

<p>В словарь <b>my_dict_user_id_liked_films</b> будут заносится пары - <b>user_id</b> и список фильмов, которые он лайкнул. <span style="color:blue";>liked_films</span> возвращает значение (список) по ключу - аргументу функции из этого словаря.</p>

In [16]:
my_dict_user_id_liked_films = {}

for i in range(length_columns):
    user_id = train_likes_df['user_id'].loc[i]
    item_id = train_likes_df['item_id'].loc[i]
    
    if user_id in my_dict_user_id_liked_films:
        my_dict_user_id_liked_films[user_id].append(item_id)
    else:
        my_dict_user_id_liked_films[user_id] = [item_id]
        
def liked_films(user_id):
    return my_dict_user_id_liked_films[user_id]

liked_films('ffd4bd3ab4e318bbaaf49ea3f5858f7d') #example (random user_id)

['79d1d57b54e299e8ecef56fd14cbb613',
 'e178fe0133cd6a93487a6223065834ab',
 'a0886341aae8fcf0d8507c4141d129b7']

<h3>Принцип работы основной функции задачи</h3>
<ul>
    <li><b>films_liked_by_user</b> присваивается список фильмов, которые лайкнул заданный <b>user</b>.</li>
    <li>Проходим циклом по этому списку, смотрим подходящее ли расстояние между фильмами (достаточно, чтобы оно было подходящим хотя бы у одного фильма из списка).</li>
    <li><b>new_film_whole_row</b> - ряд <b>dataframe</b>-а подобный тем, что создавались в функции <span style="color: blue";>dist</span>.</li>
    <li>Второй элемент этого ряда - информация о лайках на фильме - присваивается переменной <b>likes_of_new_film</b>.</li>
    <li>Среди элементов 3-12 (жанр фильма) ищется максимальный (массив из нулей и единицы). Таким способом определяется жанр фильма <b>new_film</b> и присваивается переменной <b>genre_of_new_film</b>.</li>
    <li><b>max_likes_for_new_film_genre</b> - результат работы функции <span style="color:blue";>get_max_likes</span> от жанра фильма <b>new_film</b>.</li>
    <li>Считаем вероятность, поделив количество лайков фильма на максимальное количество лайков , поставленных фильму такого же жанра.</li>
</ul>

In [17]:
dist_max = 1.1 # if distance >= dist_max we would not recommend the film at all
def user_likes_film(user, new_film):
    films_liked_by_user = liked_films(user)
    probability = 0
    
    if new_film in films_liked_by_user: #case when user has already liked this film: return 1
        probability = 1
        
        return probability
    
    for film in films_liked_by_user:
        distance = dist(film, new_film)
        if distance < dist_max:
            new_film_whole_row = df.loc[np.argmax(df['id'] == new_film)]
            
            likes_of_new_film = new_film_whole_row[1]
            genre_of_new_film = int(np.argmax(new_film_whole_row[3:12])[-1])
            max_likes_for_new_film_genre = get_max_likes(genre_of_new_film)
            
            probability = likes_of_new_film / max_likes_for_new_film_genre
            
            return probability
        
    # sometimes we'd like to randomly recommend films regardless of distance
    if random.random() < 0.01:
        probability = 1
    
    return probability

print(round(user_likes_film('b1b9adddb1932221f48fcc9e0e3574fe','3d7fdbe4e37f0a7b1409001df2212fc2'),2))

0.56


<h2>Сложности и недочеты в ходе решения задач</h2>
<ul>
    <li>Очень много времени потратил на конвертирование <b>json</b> фаила в <b>dataframe</b>. Прежде чем я понял, что вся проблема в колонках с атрибутами <b>f_N</b>, я испробовал около полудюжины разных встроенных функций/методов из разных библиотек, искал в интернете, почему вылезает такая ошибка. Из-за этих проблем вторую задачу я решал словарями.</li>

    <li>Немало также пришлось потратить на чтение документации <b>pandas</b>, поиск нужных функций (<b>concat,insert,rename</b> и др.) и случаи их применения.</li>
    
    <li>Формула подсчета вероятности далеко не идеальная.</li>
</ul>

<p>
<strong>Панель навигации:</strong><br>
<a href="#task1">Задание 1</a><br>
<a href="#task2">Задание 2</a><br>
<a href="#task3">Задание 3</a>
</p>

<p>Спасибо за внимание!<br>Весь код, включая код для построения графиков, Вы можете найти <a href="https://drive.google.com/open?id=0Bzdgm3OptRiXYktPN3ZtVnR6T3M" target="_blank">здесь</a>.</p>